In [1]:
from flask import Flask, request
import tensorflow as tf
import numpy as np
from PIL import Image

# # Load saved model
# model = tf.keras.models.load_model('my_saved_model')

# # Define Flask app
# app = Flask(__name__)

# # Define API endpoint
# @app.route('/predict', methods=['POST'])
# def predict():
#     # Get image from request
#     img_file = request.files['image']
#     img = Image.open(img_file)
    
#     # Preprocess image
#     img = img.resize((224, 224))  # resize to match input shape of model
#     img = np.asarray(img) / 255.0  # normalize pixel values
    
#     # Make prediction
#     pred = model.predict(np.expand_dims(img, axis=0))
#     pred_label = np.argmax(pred)
    
#     # Return prediction result
#     return str(pred_label)

# # Start Flask app
# if __name__ == '__main__':
#     app.run(debug=True, port=5000)

In [2]:
from werkzeug.utils import secure_filename
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Set the path to your saved model
model_path = 'history_resnet50.h5'

# Initialize the Flask app
app = Flask(__name__)

# Load the saved model
model = load_model(model_path)

# Define allowed file extensions
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/predict', methods=['POST'])
def predict():
    # Check if the POST request has a file part
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})
    
    # Get the file from the POST request
    file = request.files['file']
    
    # Check if the file has an allowed extension
    if not allowed_file(file.filename):
        return jsonify({'error': 'Invalid file extension'})
    
    # Save the file to the server
    filename = secure_filename(file.filename)
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(file_path)
    
    # Load the image and preprocess it
    img = load_img(file_path, target_size=(224, 224))
    img_arr = img_to_array(img)
    img_arr = img_arr / 255.0
    class_names = ['bed', 'chair', 'sofa']
    # Make a prediction using the loaded model
    pred = model.predict(img_arr.reshape(1, 224, 224, 3))
    class_idx = pred.argmax(axis=1)[0]
    class_label = class_names[class_idx]
    
    # Return the predicted class label
    return jsonify({'class_label': class_label})

if __name__ == '__main__':
    app.run()

In [6]:
app = Flask(__name__)

# Load the saved model
model = load_model('history_resnet50.h5')

# Define the API endpoint
@app.route('/predict', methods=['POST'])
# def predict():
#     # Get the image file from the request
#     file = request.files['file']

#     # Convert the image file to a numpy array
#     img = image.img_to_array(image.load_img(file, target_size=(224, 224))) / 255.0
#     img = np.expand_dims(img, axis=0)

#     # Get the prediction
#     pred = model.predict(img)[0]

#     # Get the label of the prediction
#     label = 'positive' if pred >= 0.5 else 'negative'

#     # Return the prediction as a JSON object
#     return jsonify({'prediction': label})
def predict():
    # get the image file from the request
    file = request.files['file']

    # read the image file
    img = Image.open(io.BytesIO(file.read()))

    # preprocess the image
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # make the prediction
    pred = model.predict(img_array)

    # return the prediction result
    result = {'class': str(np.argmax(pred))}
    return jsonify(result)

try:
    if __name__ == '__main__':
        app.run(debug=True)
except Exception as e:
    %tb


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1